In [249]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import numpy as np
from sklearn.preprocessing import StandardScaler


In [250]:
import warnings
warnings.filterwarnings("ignore")

In [318]:
data = pd.read_csv('facies_vectors.csv')

In [319]:
feature_names = ['GR', 'ILD_log10', 'DeltaPHI', 'PHIND', 'PE', 'NM_M', 'RELPOS']
facies_names = ['SS', 'CSiS', 'FSiS', 'SiSh', 'MS', 'WS', 'D', 'PS', 'BS']
facies_colors = ['#F4D03F', '#F5B041','#DC7633','#6E2C00', '#1B4F72','#2E86C1', '#AED6F1', '#A569BD', '#196F3D']

In [320]:
data = data.fillna(data['PE'].mean())
test = dict()
train = dict()
for well in set(data['Well Name']):
    test[well] = data[data['Well Name'] == well]
    train[well] = data[data['Well Name'] != well]

In [263]:
data = data.fillna(data['PE'].mean())
train, test = train_test_split(data, test_size=0.3)

In [202]:
def take_n_elements(data, key, index, window):
    return [data[key][element] for element in range(index, index + window)]

In [204]:
def transform_data(data, window):
    new_data = pd.DataFrame()
    data.sort_values(by=['Depth'])
    for key in data:
        new_data[key] = [take_n_elements(data, key, index, window) for index in range(0, len(data), window) ]

In [ ]:
Window_widths = [1, 2, 3, 4, 5]
transformed_train = dict()
for well in set(train['Well Name']):
    for window in Window_widths:
        transformed_train[window] = transform_data(train[train['Well Name'] == well], window)

In [321]:
def accuracy(y_predict, y_test):
    truth = [y_predict[index] for index in range(len(y_predict)) if y_predict[index] == y_test[index]]
    return len(truth)/len(y_predict)

In [322]:
import xgboost as xgb
from xgboost.sklearn import  XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from keras.models import Sequential
from keras.layers import Dense
def classification(clf):
    acc= 0
    for iter in range(10):
        clf.fit(X_train_robust_norm, y_train)
        y_predict = clf.predict(X_test_robust_norm)
        acc += accuracy(y_predict, y_test)
    acc /= 10
    print('With robust and norm',acc)
    return acc

In [326]:
global_acc = 0
for well in set(data['Well Name']):
    X_train = train[well][feature_names].values 
    y_train = train[well]['Facies'].values 
    X_test = test[well][feature_names].values 
    y_test = test[well]['Facies'].values 
    well_train = train[well]['Well Name'].values
    well_test = test[well]['Well Name'].values
    depth_train = train[well]['Depth'].values
    depth_test = test[well]['Depth'].values
    
    robust = preprocessing.RobustScaler(quantile_range=(25.0, 75.0)).fit(X_train)
    X_train_robust = robust.transform(X_train)
    X_test_robust = robust.transform(X_test)
    
    scaler = StandardScaler().fit(X_train_robust)
    X_train_robust_norm = scaler.transform(X_train_robust)
    X_test_robust_norm = scaler.transform(X_test_robust)

    clf_xgb = XGBClassifier(learning_rate=0.12, max_depth=3,n_estimators=150)
    global_acc += classification(clf_xgb)
    print(global_acc/10)

With robust and norm 0.47516198704103674
0.047516198704103674
With robust and norm 0.5965665236051503
0.10717285106461868
With robust and norm 0.373253493013972
0.14449820036601588
With robust and norm 0.7249999999999999
0.21699820036601586
With robust and norm 0.5605095541401274
0.2730491557800286
With robust and norm 0.5726681127982648
0.33031596705985505
With robust and norm 0.5519801980198019
0.38551398686183525
With robust and norm 0.5079726651480637
0.4363112533766416
With robust and norm 0.48329621380846327
0.48464087475748796
With robust and norm 0.4819277108433734
0.5328336458418252


In [306]:
X_train.shape

(3187, 7)

In [307]:
robust = preprocessing.RobustScaler(quantile_range=(25.0, 75.0)).fit(X_train)
X_train_robust = robust.transform(X_train)
X_test_robust = robust.transform(X_test)

scaler = StandardScaler().fit(X_train_robust)
X_train_robust_norm = scaler.transform(X_train_robust)
X_test_robust_norm = scaler.transform(X_test_robust)

scaler = StandardScaler().fit(X_train)
X_train_norm = scaler.transform(X_train)
X_test_norm = scaler.transform(X_test)

robust = preprocessing.RobustScaler(quantile_range=(25.0, 75.0)).fit(X_train_norm)
X_train_norm_robust = robust.transform(X_train_norm)
X_test_norm_robust = robust.transform(X_test_norm)



In [310]:
clf_xgb = XGBClassifier(learning_rate=0.12, max_depth=3,n_estimators=150)
classification(clf_xgb)

With robust and norm 0.40718562874251496


In [311]:
clf_log = LogisticRegression(multi_class='ovr')
classification(clf_log)


With robust and norm 0.469061876247505


In [312]:
clf_per = MLPClassifier(hidden_layer_sizes=(100,100,100))
classification(clf_per)

With robust and norm 0.4520958083832335


In [15]:
from keras.utils import to_categorical
y_train_tocat = to_categorical(y_train, 10)
y_test_tocat = to_categorical(y_test, 10)
model = Sequential()
model.add(Dense(100, input_dim=7, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(10, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train_robust_norm, y_train_tocat, validation_split=0.3, epochs=200, batch_size=10)
model.evaluate(X_test_robust_norm, y_test_tocat)

Train on 2032 samples, validate on 872 samples
Epoch 1/200
2032/2032 [==============================] - 1s 602us/step - loss: 1.4418 - acc: 0.4419 - val_loss: 1.1382 - val_acc: 0.5321
Epoch 2/200
2032/2032 [==============================] - 1s 401us/step - loss: 1.0839 - acc: 0.5595 - val_loss: 1.0559 - val_acc: 0.5757
Epoch 3/200
2032/2032 [==============================] - 1s 399us/step - loss: 1.0204 - acc: 0.5738 - val_loss: 1.0351 - val_acc: 0.5872
Epoch 4/200
2032/2032 [==============================] - 1s 414us/step - loss: 0.9801 - acc: 0.5984 - val_loss: 1.0218 - val_acc: 0.5883
Epoch 5/200
2032/2032 [==============================] - 1s 412us/step - loss: 0.9572 - acc: 0.6038 - val_loss: 0.9976 - val_acc: 0.5975
Epoch 6/200
2032/2032 [==============================] - 1s 405us/step - loss: 0.9268 - acc: 0.6147 - val_loss: 0.9828 - val_acc: 0.6170
Epoch 7/200
2032/2032 [==============================] - 1s 421us/step - loss: 0.9095 - acc: 0.6289 - val_loss: 0.9733 - val_acc: 0

2032/2032 [==============================] - 1s 414us/step - loss: 0.3608 - acc: 0.8469 - val_loss: 1.1219 - val_acc: 0.6640
Epoch 61/200
2032/2032 [==============================] - 1s 400us/step - loss: 0.3622 - acc: 0.8538 - val_loss: 1.1036 - val_acc: 0.6743
Epoch 62/200
2032/2032 [==============================] - 1s 409us/step - loss: 0.3439 - acc: 0.8647 - val_loss: 1.0935 - val_acc: 0.6766
Epoch 63/200
2032/2032 [==============================] - 1s 405us/step - loss: 0.3407 - acc: 0.8612 - val_loss: 1.1531 - val_acc: 0.6835
Epoch 64/200
2032/2032 [==============================] - 1s 399us/step - loss: 0.3356 - acc: 0.8602 - val_loss: 1.1496 - val_acc: 0.6823
Epoch 65/200
2032/2032 [==============================] - 1s 376us/step - loss: 0.3287 - acc: 0.8696 - val_loss: 1.1381 - val_acc: 0.6892
Epoch 66/200
2032/2032 [==============================] - 1s 402us/step - loss: 0.3317 - acc: 0.8745 - val_loss: 1.1298 - val_acc: 0.6881
Epoch 67/200
2032/2032 [=======================

2032/2032 [==============================] - 1s 400us/step - loss: 0.1074 - acc: 0.9675 - val_loss: 2.1094 - val_acc: 0.6789
Epoch 179/200
2032/2032 [==============================] - 1s 429us/step - loss: 0.1677 - acc: 0.9562 - val_loss: 2.1435 - val_acc: 0.6743
Epoch 180/200
2032/2032 [==============================] - 1s 412us/step - loss: 0.2293 - acc: 0.9291 - val_loss: 2.0666 - val_acc: 0.6904
Epoch 181/200
2032/2032 [==============================] - 1s 414us/step - loss: 0.1123 - acc: 0.9656 - val_loss: 2.1346 - val_acc: 0.6950
Epoch 182/200
2032/2032 [==============================] - 1s 409us/step - loss: 0.1107 - acc: 0.9636 - val_loss: 2.1475 - val_acc: 0.6709
Epoch 183/200
2032/2032 [==============================] - 1s 421us/step - loss: 0.0865 - acc: 0.9754 - val_loss: 2.0536 - val_acc: 0.6915
Epoch 184/200
2032/2032 [==============================] - 1s 411us/step - loss: 0.0681 - acc: 0.9808 - val_loss: 2.0516 - val_acc: 0.6858
Epoch 185/200
2032/2032 [================

[1.9645717413071169, 0.6899598386871767]

In [ ]:
acc = 0
    for iter in range(10):
        clf.fit(X_train_robust, y_train)
        y_predict = clf.predict(X_test_robust)
        acc += accuracy(y_predict, y_test)
    acc /= 10
    print('With robust',acc)
    acc= 0
    for iter in range(10):
        clf.fit(X_train, y_train)
        y_predict = clf.predict(X_test)
        acc += accuracy(y_predict, y_test)
    acc /= 10
    print('without anything',acc)
    acc= 0
    for iter in range(10):
        clf.fit(X_train_robust_norm, y_train)
        y_predict = clf.predict(X_test_robust_norm)
        acc += accuracy(y_predict, y_test)
    acc /= 10
    print('With robust and norm',acc)
    for iter in range(10):
        clf.fit(X_train_norm, y_train)
        y_predict = clf.predict(X_test_norm)
        acc += accuracy(y_predict, y_test)
    acc /= 10
    print('With norm',acc)
    acc = 0
    for iter in range(10):
        clf.fit(X_train_norm_robust, y_train)
        y_predict = clf.predict(X_test_norm_robust)
        acc += accuracy(y_predict, y_test)
    acc /= 10
    print('With norm and robust',acc)
    'NM_M'